# setting

In [1]:
# import libraries
import mne, os, pickle, glob, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as stats
from mne.stats import spatio_temporal_cluster_1samp_test

In [2]:
# directories
mri_dir  = os.path.realpath('../Data/mri')           # mri directory
meg_dir  = os.path.realpath('../Data/meg-localizer') # meg directory
log_dir  = os.path.join(meg_dir, 'log')              # log directory
stc_dir  = os.path.join(meg_dir, 'stc')              # stc directory
mod_dir  = os.path.join(meg_dir, 'mod')              # GLM model directory
res_dir  = os.path.join(meg_dir, 'res')              # results directory

# subject list
subjects = [
    'P049','P050','P054','P056','P057',
    'P058','P060','P061','P062','P063',
    'P064','P065','P066','P068','P069',
    'P070','P071'
    ]

# define anatomical mask: bilateral ventral temporo-occipital regions
Following the approach in [Gwilliam et al. (2016) *NeuroImage*](https://doi.org/10.1016/j.neuroimage.2016.02.057), the anatomical mask includes the following brain regions from the Desikan-Killiany Atlas:
- lateraloccipital
- cuneus
- lingual
- pericalcarine
- fusiform
- middle-temporal
- inferior-temporal

In [3]:
# read labels
labels = mne.read_labels_from_annot('fsaverage', 'aparc', 'both', subjects_dir=mri_dir)

# define ROIs
roi_list = ['lateraloccipital', 'cuneus', 'lingual', 'pericalcarine', 
            'fusiform', 'middletemporal', 'inferiortemporal']

# loop over ROI list
for roi_count, roi_name in enumerate(roi_list):
    
    # look for ROI info from the annotation labels 
    roi_lh_tmp = [label for label in labels if label.name == f'{roi_name}-lh'][0]
    roi_rh_tmp = [label for label in labels if label.name == f'{roi_name}-rh'][0]
    
    # combine ROIs
    if roi_count == 0:
        roi_lh = roi_lh_tmp
        roi_rh = roi_rh_tmp
    else:
        roi_lh += roi_lh_tmp
        roi_rh += roi_rh_tmp

# set up vertex indices
n_hemisources = 2562  # number of vertices in each hemisphere
hemi_idx = np.arange(0, n_hemisources, 1)  # create an array

# look for ROI vertex indices in the LH, RH, and both himispheres
roi_idx_lh = roi_lh.get_vertices_used(vertices=hemi_idx)
roi_idx_rh = roi_rh.get_vertices_used(vertices=hemi_idx)
roi_idx_bh = np.concatenate((roi_idx_lh, roi_idx_rh+n_hemisources))

# look for ROI vertex indices NOT in the LH, RH, and both himispheres
diff_idx_lh = np.setdiff1d(hemi_idx, roi_idx_lh)
diff_idx_rh = np.setdiff1d(hemi_idx, roi_idx_rh)
diff_idx_bh = np.concatenate((diff_idx_lh, diff_idx_rh+n_hemisources))

Reading labels from parcellation...
   read 35 labels from /Users/cl5564/Library/CloudStorage/Dropbox/OSF/TagalogViolation/Data/mri/fsaverage/label/lh.aparc.annot
   read 34 labels from /Users/cl5564/Library/CloudStorage/Dropbox/OSF/TagalogViolation/Data/mri/fsaverage/label/rh.aparc.annot


# load beta map
- Model: 'constant', 'word_clean', 'word_noisy', 'word_symbol', 'letter_clean', 'letter_noisy', 'letter_symbol', 'trial order'

In [4]:
# define event codes
event_id  = dict(word_clean   = 1, word_noisy   = 2, word_symbol  = 4,
                 letter_clean = 8, letter_noisy = 9, letter_symbol = 3)
cond_code = list(event_id.values())  # condition code
cond_name = list(event_id.keys())    # condition name

# define regressors (8 regressors)
# constant + 6 conditions + trial order
col_labels = cond_name.copy()
col_labels.insert(0, 'constant')
for item in ['trial order']: col_labels.append(item)

# define variables
epoch_tmin = -0.1  # epoch onset
epoch_tmax = 0.6   # epoch offset
times      = np.arange(epoch_tmin*1000, epoch_tmax*1000+1, 1)
n_subj     = len(subjects)    # number of subjects
n_cond     = len(cond_code)   # number of conditions
n_reg      = len(col_labels)  # number of regressors
n_times    = len(times)       # numner of time points
n_sources  = n_hemisources*2  # number of sources

# create an empty matrix for storing beta map
data_mtx  = np.empty((n_subj, n_sources, n_times, n_reg))

# load beta map
for s, subj in enumerate(subjects):
    print(subj, end=' ')
    fname    = os.path.join(mod_dir, '%s_reg%s_b-map.npy') %(subj, n_reg)
    data_val = np.load(fname)
    data_mtx[s,:,:,:] = data_val

data_mtx = data_mtx[:,:,:,1:7] # keep condition regressors
print();print(data_mtx.shape)

P049 P050 P054 P056 P057 P058 P060 P061 P062 P063 P064 P065 P066 P068 P069 P070 P071 
(17, 5124, 701, 6)


# String effect
- contrast: 4-unit word > 4-unit symbol

In [5]:
# define time of interest
toi_min = 130
toi_max = 180
toi     = np.arange(toi_min, toi_max+1, 1)
toi_min_idx = np.squeeze(np.where(times==toi_min))
toi_max_idx = np.squeeze(np.where(times==toi_max))
toi_idx     = np.arange(toi_min_idx,toi_max_idx+1)
toi_ntimes  = toi_idx.size

# conditions: word_clean, word_noisy, word_symbol, letter_clean, letter_noisy, letter_symbol
# contrast: 4-unit word > 4-unit symbol
contrast = [1, 0, -1, 0, 0, 0]
data = np.dot(data_mtx, contrast)
data = np.transpose(data, [0, 2, 1]) # transpose the matrix into: subjects x times x sources
data_bh = data[:,toi_idx,:]
data_lh = data[:,toi_idx,:n_hemisources]
data_rh = data[:,toi_idx,n_hemisources:]

In [6]:
# define parameters
hemi            = 'bh'
X               = eval('data_%s'%hemi)
spatial_exclude = eval('diff_idx_%s'%hemi)
tail            = 1
p_thresh        = 0.05
n_permutations  = 10000
df              = n_subj-1

# read source space & compute adjacency
src_fname = os.path.join(mri_dir, 'fsaverage', 'bem', 'fsaverage-ico-4-src.fif')
src = mne.read_source_spaces(src_fname)
if hemi == 'bh':
    adjacency = mne.spatial_src_adjacency(src)
elif hemi == 'lh':
    adjacency = mne.spatial_src_adjacency(src[:1])
elif hemi == 'rh':
    adjacency = mne.spatial_src_adjacency(src[1:])

# define threshold
if tail == -1:
    t_thresh = -stats.distributions.t.ppf(1-p_thresh, df=df)
    threshold_tfce = dict(start=np.round(t_thresh), step=0.1*tail)
else:
    t_thresh = stats.distributions.t.ppf(1-p_thresh, df=df)
    threshold_tfce = dict(start=np.round(t_thresh), step=0.1)

# permutation test with TFCE
print('permutation test...')
t_tfce, clusters, p_tfce, H0 = clu = \
    spatio_temporal_cluster_1samp_test(X,
                                       tail            = tail,
                                       threshold       = threshold_tfce,
                                       n_permutations  = n_permutations,
                                       adjacency       = adjacency, 
                                       spatial_exclude = spatial_exclude,
                                       n_jobs          = 10,
                                       seed            = 1119,
                                       buffer_size     = None,
                                       verbose         = True)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124
permutation test...
stat_fun(H1): min=-5.994030 max=6.725490
Running initial clustering …
Using 48 thresholds from 2.00 to 6.70 for TFCE computation (h_power=2.00, e_power=0.50)
Found 261324 clusters


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

In [7]:
p_thresh = 0.05
pval = np.reshape(p_tfce, [toi_ntimes, n_sources])
if tail == -1:
    pval_cmp = pval > 1-p_thresh
else:
    pval_cmp = pval < p_thresh
    
vidx = np.unique(np.where(pval_cmp)[1])
tidx = np.unique(np.where(pval_cmp)[0])
clu_tmin  = toi[tidx[0]]
clu_tmax  = toi[tidx[-1]]
clu_times = np.arange(clu_tmin, clu_tmax+1)

if clu_times.size == tidx.size:
    print('time range: %s-%sms' %(clu_tmin, clu_tmax))
    print('LH vertices: %s' %(sum(vidx < n_hemisources)))
    print('RH vertices: %s' %(sum(vidx >= n_hemisources)))
else:
    print('time range is not continuous...')

time range: 130-170ms
LH vertices: 61
RH vertices: 0


In [8]:
# save permutation test results
current_date  = datetime.date.today()
contrast_name = 'reg%s_string' %(n_reg)
pickle_fname  = os.path.join(res_dir, 'res_%s_%s-%s_%ssubjs_tfce_%s.pickled') % (contrast_name, toi_min, toi_max, n_subj, current_date)
open_file     = open(pickle_fname, 'wb')
pickle.dump(clu, open_file)
open_file.close()

# Noise effect
- contrast: (noisy word + noisy letter) > (clean word + clean letter)

In [9]:
# orgaize data
toi_min = 80
toi_max = 130
toi     = np.arange(toi_min, toi_max+1, 1)
toi_min_idx = np.squeeze(np.where(times==toi_min))
toi_max_idx = np.squeeze(np.where(times==toi_max))
toi_idx     = np.arange(toi_min_idx,toi_max_idx+1)
toi_ntimes  = toi_idx.size

# conditions: word_clean, word_noisy, word_symbol, letter_clean, letter_noisy, letter_symbol
# contrast: (noisy word + noisy letter) > (clean word + clean letter)
contrast = [-0.5, 0.5, 0, -0.5, 0.5, 0]
data = np.dot(data_mtx, contrast)
data = np.transpose(data, [0, 2, 1]) # transpose the matrix into: subjects x times x sources
data_bh = data[:,toi_idx,:]
data_lh = data[:,toi_idx,:n_hemisources]
data_rh = data[:,toi_idx,n_hemisources:]

In [10]:
# define parameters
hemi            = 'bh'
X               = eval('data_%s'%hemi)
spatial_exclude = eval('diff_idx_%s'%hemi)
tail            = -1
p_thresh        = 0.05
n_permutations  = 10000
df              = n_subj-1

# read source space & compute adjacency
src_fname = os.path.join(mri_dir, 'fsaverage', 'bem', 'fsaverage-ico-4-src.fif')
src = mne.read_source_spaces(src_fname)
if hemi == 'bh':
    adjacency = mne.spatial_src_adjacency(src)
elif hemi == 'lh':
    adjacency = mne.spatial_src_adjacency(src[:1])
elif hemi == 'rh':
    adjacency = mne.spatial_src_adjacency(src[1:])

# define threshold
if tail == -1:
    t_thresh = -stats.distributions.t.ppf(1-p_thresh, df=df)
    threshold_tfce = dict(start=np.round(t_thresh), step=0.1*tail)
else:
    t_thresh = stats.distributions.t.ppf(1-p_thresh, df=df)
    threshold_tfce = dict(start=np.round(t_thresh), step=0.1)

# permutation test with TFCE
print('permutation test...')
t_tfce, clusters, p_tfce, H0 = clu = \
    spatio_temporal_cluster_1samp_test(X,
                                       tail            = tail,
                                       threshold       = threshold_tfce,
                                       n_permutations  = n_permutations,
                                       adjacency       = adjacency, 
                                       spatial_exclude = spatial_exclude,
                                       n_jobs          = 10,
                                       seed            = 1119,
                                       buffer_size     = None,
                                       verbose         = True)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124
permutation test...
stat_fun(H1): min=-6.526754 max=6.221838
Running initial clustering …
Using 46 thresholds from -2.00 to -6.50 for TFCE computation (h_power=2.00, e_power=0.50)
Found 261324 clusters


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

In [11]:
p_thresh = 0.05
pval = np.reshape(p_tfce, [toi_ntimes, n_sources])
if tail == -1:
    pval_cmp = pval > 1-p_thresh
else:
    pval_cmp = pval < p_thresh
    
vidx = np.unique(np.where(pval_cmp)[1])
tidx = np.unique(np.where(pval_cmp)[0])
clu_tmin  = toi[tidx[0]]
clu_tmax  = toi[tidx[-1]]
clu_times = np.arange(clu_tmin, clu_tmax+1)

if clu_times.size == tidx.size:
    print('time range: %s-%sms' %(clu_tmin, clu_tmax))
    print('LH vertices: %s' %(sum(vidx < n_hemisources)))
    print('RH vertices: %s' %(sum(vidx >= n_hemisources)))
else:
    print('time range is not continuous...') 

time range: 80-130ms
LH vertices: 87
RH vertices: 78


In [12]:
# save permutation test results
current_date  = datetime.date.today()
contrast_name = 'reg%s_noise' %(n_reg)
pickle_fname  = os.path.join(res_dir, 'res_%s_%s-%s_%ssubjs_tfce_%s.pickled') % (contrast_name, toi_min, toi_max, n_subj, current_date)
open_file     = open(pickle_fname, 'wb')
pickle.dump(clu, open_file)
open_file.close()